In [1]:
test_simple = "Racontez-moi une bref histoire"
test_comparison = """
       Comparez la description donnée à d'autres descriptions, formatées en JSON, et donnez-moi l'ID du projet qui est le plus similaire au target_project. 
       Ne considérez que la similarité entre les descriptions.
       target_project : {"id": 1, "description": "Développement d'une application mobile pour la gestion des tâches personnelles avec rappels et synchronisation cloud."}
       Descriptions des autres projets :
       [
       {"id": 4, "description": "Création d'une application mobile de gestion des tâches personnelles, intégrant rappels automatiques et synchronisation entre appareils."},
         {"id": 3, "description": "Mise en place d'un système web pour la gestion de projets collaboratifs avec calendrier partagé."},
         {"id": 2, "description": "Développement d'un jeu vidéo éducatif pour enfants en bas âge, avec des animations interactives."}
       ]
       Répondez avec une liste d'ID des projets similaires, classés par ordre de similarité, du plus similaire au moins similaire.
       """
test_real_case = """
       Comparez la description donnée à d'autres descriptions, formatées en JSON, et donnez-moi l'ID du projet qui est le plus similaire au target_project. 
       Ne considérez que la similarité entre les descriptions.
       target_project : 
       {'id': 95, 'description': "Les murs de cette école neuve sont en béton brut d'aspect très austère, genre enceinte d'une prison, et ils emmagasinent la chaleur. Une misérable plate bande de végétation basse (déjà envahie par les mauvaises herbes ), les sépare de la voie publique. Nous proposons d'habiller ces murs de plantes grimpantes adaptées en fonction de l'exposition et, la où ce n'est pas possible, (au droit des ouvertures) de réaliser des plantations de 4ème grandeur (5 m maxi) cela permettra aussi de prévenir les tags et autres graffitis ... Prévoir aussi des gites pour les oiseaux cavernicoles et les chauve-souris Cet aménagement concerne en particulier l'allée de grand selve, la rue Simone Henry et la rue Flora Tristan"}
       
       
       Descriptions des autres projets :
       [{'id': 99, 'description': 'Cette idée a été transmise via un formulaire papier par molac26. Au lieu de planter des arbres ornementaux, plantons des arbres fruitiers à récolter par les écoliers pour les cantines scolaires et des associations volontaires. Avantages : fruit bio pour les cantines / découverte des fruits par les enfants / Baisse du coût des cantines / Alimentation locale Inconvénients : planter des arbres fruitiers'}, {'id': 93, 'description': 'La pollution sonore induite par la densification du quartier devient très problématique et un obstacle au bien-vivré ensemble. Les 2-roues en sont la principale source (vitesse excessive, moteurs et échappements trafiqués). Serait-il possible de mener une campagne de sensibilisation et installer des radars sonores (Chemin des izards, route de Launaguet…) ?'}, {'id': 92, 'description': 'Le parking de l’impasse du cimetière de Croix Daurade est une grande flaque de bitume; cela avait du sens quand il accueillait les bus du lycée mais aujourd’hui il est venu le temps de verdir cette flaque de bitume: dalles engazonnées de stationnement et arbres 🌳 ferait un grand bien à l’environnement de notre quartier.'}, {'id': 201, 'description': "Plusieurs jeux à destination des tout petits ou plus grands sont en plein soleil à Toulouse: - Place Saint Georges - Place Occitane - Raymond VI - Borderouge -Jardin Niel D'autres exemples sont possibles ! Tobogans brulants, coups de chaud.... ils sont impraticables... La mairie pourrait assez facilement installer des voilages suspendus en hauteur accrochés à des piquets comme cela se pratique au Portugal ou en Espagne afin de les rendre ombragés."}, {'id': 91, 'description': "A l'instar d'autres communes de la métropole, il serait utile d'envisager une participation de la mairie au coûts de nichoirs à martinets et chauve-souris (une commande massive par la mairie permettant aussi de faire baisser les prix à l'unité) pour une lutte collective contre le moustique tigre. Ces nichoirs seraient distribués aux particuliers qui les placeraient à leur domicile. Il pourrait être envisagé également, sur le même principe, une participation à des pièges à larves ou à adultes de moustiques tigres."}, {'id': 94, 'description': "Le quartier de Croix Daurade fait parti de ceux durement touchés par l'arrivée des moustiques tigres. Peut-être serait-il possible d'installer des nichoirs à chauve souris, des pièces à moustiques dans le quartier ?"}, {'id': 98, 'description': 'Je souhaite des équipements sportifs de type "musculation" en plein air, de libre accès, installés soit le long des sentes piétonnières, soit dans les Jardins du Museum, donc dans la verdure et à l\'ombre. Quartier TROIS COCUS - BORDEROUGE'}]


       
       Répondez avec une liste d'ID des projets similaires, classés par ordre de similarité, du plus similaire au moins similaire.
       """

### Croissant LLM 

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
model_name = "croissantllm/CroissantLLMChat-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
generation_args = {
    "max_new_tokens": 256,
    "do_sample": True,
    "temperature": 0.3,
    "top_p": 0.90,
    "top_k": 40,
    "repetition_penalty": 1.05,
    "eos_token_id": [tokenizer.eos_token_id, 32000],
}

chat = [
   {
       "role": "user", 
       "content": """
       Comparez la description donnée à d'autres descriptions, formatées en JSON, et donnez-moi l'ID du projet qui est le plus similaire au target_project. 
       Ne considérez que la similarité entre les descriptions.
       target_project : {"id": 1, "description": "Développement d'une application mobile pour la gestion des tâches personnelles avec rappels et synchronisation cloud."}
       Descriptions des autres projets :
       [
       {"id": 4, "description": "Création d'une application mobile de gestion des tâches personnelles, intégrant rappels automatiques et synchronisation entre appareils."},
         {"id": 3, "description": "Mise en place d'un système web pour la gestion de projets collaboratifs avec calendrier partagé."},
         {"id": 2, "description": "Développement d'un jeu vidéo éducatif pour enfants en bas âge, avec des animations interactives."}
       ]
       Répondez avec une liste d'ID des projets similaires, classés par ordre de similarité, du plus similaire au moins similaire.
       """
   },
]

chat_input = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

inputs = tokenizer(chat_input, return_tensors="pt").to(model.device)
tokens = model.generate(**inputs, **generation_args)

print(tokenizer.decode(tokens[0]))
# print tokens individually
# print([(tokenizer.decode([tok]), tok) for tok in tokens[0].tolist()])

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.
Exception ignored in: <function Group.__del__ at 0x762f03877b00>
Traceback (most recent call last):
  File "/home/samuel-malaga/Documents/Master2IS/Stage M1/M2IS-Internship-XAI/env/lib/python3.12/site-packages/regex/_regex_core.py", line 3035, in __del__
    def __del__(self):

KeyboardInterrupt: 


KeyboardInterrupt: 

#### RESULTS: Horrible performance, to run simple prompts, it took more than 5 minutes, unusable

### French alpaca 3b 

In [ ]:
from ollama import chat

In [ ]:
alpaca_3b = chat(
    messages = [
        {
            'role': 'user',
            'content': test_real_case
        }
    ],
    model='jpacifico/french-alpaca-3b:latest'
    # ,
    # format = ResponseFormatter.model_json_schema(),
)
# ResponseFormatter.model_validate_json()
similarity_response = alpaca_3b.message.content
print(similarity_response)

Weird results in the test_comparison and test real case, I'm assuming that it won't work in a pipeline 

### Instruct French

In [ ]:
instruct_french = chat(
    messages = [
        {
            'role': 'user',
            'content': test_real_case
        }
    ],
    model='Volko76/llama3.2-instruct-french:1b'
    # ,
    # format = ResponseFormatter.model_json_schema(),
)

similarity_response = instruct_french.message.content
print(similarity_response)

It worked kind of fast with the test_real_case, the output wasn't the best (compared to what we can get out of GPT) but it worked without any major flaw, maybe this model will be good for the entity extraction.